In [ ]:
%%capture
!pip install transformers==4.40.2
!pip install simpletransformers

In [ ]:
# Import necessary libraries
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from simpletransformers.classification import ClassificationModel
from sklearn.metrics import accuracy_score
import numpy as np
from scipy.special import softmax
from google.colab import files

In [ ]:
uploaded = files.upload()

Saving merged_depression.csv to merged_depression.csv


Function for calculate metrics

In [ ]:
def metrics(result):
  tp_fp = result["tp"] + result["fp"]
  tp_fn = result["tp"] + result["fn"]
  tn_fp = result["tn"] + result["fp"]

  # Calculate metrics
  accuracy = result["accuracy"]
  precision = result["tp"] / (tp_fp) if (tp_fp) > 0 else 0    # Positive Predictive Value
  recall = result["tp"] / (tp_fn) if (tp_fn) > 0 else 0       # Sensitivity, True Positive Rate)
  f1 = (2*precision*recall)/(precision+recall)

  print(f'Accuracy: {accuracy}', f'Precision: {precision}', f'Recall: {recall}', f'F1 Score: {f1}')

model configuration

In [ ]:
# Model configuration
model_args = {"num_train_epochs": 10, "overwrite_output_dir": True, 'max_seq_length':255,
              'use_early_stopping':True, 'early_stopping_delta':0.001, "manual_seed": 42}
cuda_available = torch.cuda.is_available()

# Create a ClassificationModel
model = ClassificationModel("bert", "bert-base-cased", args=model_args, use_cuda=cuda_available)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

Import of the two datasets

In [ ]:
# Read the two datasets
df = pd.read_csv('/content/processed_depression_MAIN_data_SMOTE.csv')
df2 = pd.read_csv('/content/depression_RSDD.csv')

In [ ]:
# Used one dataset as training and the other for test
train_df = df
val_df = df2

# Print the shapes of the train and validation datasets
print(f'Training Dataset Shape: {train_df.shape}')
print(f'Validation Dataset Shape: {val_df.shape}')

Training Dataset Shape: (23938, 2)
Validation Dataset Shape: (7731, 2)


In [ ]:
# Train the model
model.train_model(train_df)

/usr/local/lib/python3.10/dist-packages/simpletransformers/classification/classification_model.py:610: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


  0%|          | 0/47 [00:00<?, ?it/s]

Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Running Epoch 1 of 10:   0%|          | 0/2993 [00:00<?, ?it/s]

Running Epoch 2 of 10:   0%|          | 0/2993 [00:00<?, ?it/s]

...

Running Epoch 7 of 10:   0%|          | 0/2993 [00:00<?, ?it/s]

Running Epoch 8 of 10:   0%|          | 0/2993 [00:00<?, ?it/s]

Running Epoch 9 of 10:   0%|          | 0/2993 [00:00<?, ?it/s]

Running Epoch 10 of 10:   0%|          | 0/2993 [00:00<?, ?it/s]

(29930, 0.5267021013867524)

In [ ]:
# Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(val_df)

/usr/local/lib/python3.10/dist-packages/simpletransformers/classification/classification_model.py:1453: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


  0%|          | 0/15 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/78 [00:00<?, ?it/s]

In [ ]:
# Convert logits in probabilities using softmax function
probabilities = softmax(model_outputs, axis=1)
# Predict the classes using the index with the highest probability
predictions = np.argmax(probabilities, axis=1)

In [ ]:
# number of wrong predictions
print(len(wrong_predictions))

1904


In [ ]:
# metrics
metrics(result)

Accuracy: 0.47445349890053035 Precision: 0.43862433862433864 Recall: 0.21639258679196033 F1 Score: 0.2898094738682048
